In [74]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = 50

In [117]:
final_train_df = pd.read_csv('cleaned_train_set.csv')
final_test_df = pd.read_csv('cleaned_test_set.csv')
df_train =pd.read_csv('train_v9rqX0R.csv')

In [118]:
X_train_df = final_train_df.iloc[:, [0,3,4,28,29,30,31,32,33,34,35,36,37]].copy()
X_test_df = final_test_df.iloc[:, [0,3,4,28,29,30,31,32,33,34,35,36,37]].copy()
Y_train_all = df_train['Item_Outlet_Sales']

In [119]:
from sklearn.preprocessing import MinMaxScaler

minmax = MinMaxScaler(feature_range=(0,1))

In [120]:
X_train_df[['Item_MRP','Item_Visibility']] = minmax.fit_transform(X_train_df[['Item_MRP','Item_Visibility']])
X_test_df[['Item_MRP','Item_Visibility']] = minmax.transform(X_test_df[['Item_MRP','Item_Visibility']])

In [121]:
items_in_train = X_train_df['Item_Identifier'].unique().tolist()
items_in_test = X_test_df['Item_Identifier'].unique().tolist()

items_missing_in_test = [x for x in items_in_train if x not in items_in_test]

In [122]:
X_train_df = pd.get_dummies(X_train_df, columns = ['Item_Identifier'])
X_test_df = pd.get_dummies(X_test_df, columns = ['Item_Identifier'])

In [123]:
for col in items_missing_in_test:
    X_test_df["Item_Identifier_" + col] = np.zeros(5681)

X_test_df = X_test_df.sort_index(axis = 1)
X_train_df = X_train_df.sort_index(axis = 1)

In [125]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor

In [126]:
X_train, X_val, y_train, y_val = train_test_split(X_train_df, Y_train_all, test_size=0.2, random_state=13)

In [127]:
param_grid = {
    'max_depth' : np.arange(2,50)
} 
gbr_gscv = DecisionTreeRegressor()
gbr_gscv = GridSearchCV(gbr_gscv, param_grid=param_grid, n_jobs=1, scoring='neg_mean_absolute_error', cv=5)
gbr_gscv.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeRegressor(), n_jobs=1,
             param_grid={'max_depth': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])},
             scoring='neg_mean_absolute_error')

In [128]:
print(gbr_gscv.best_params_)
print(gbr_gscv.best_score_)

{'max_depth': 7}
-765.7480982363902


Decision Tree RandomizedSearchCV results:

Mean Absolute Error:'splitter': 'random', 'min_weight_fraction_leaf': 0.1, 'min_samples_leaf': 6, 'max_leaf_nodes': 40, 'max_features': None, 'max_depth': 1

Mean Squared Error:'splitter': 'best', 'min_weight_fraction_leaf': 0.1, 'min_samples_leaf': 4, 'max_leaf_nodes': 60, 'max_features': None, 'max_depth': 7

In [153]:
Dt = DecisionTreeRegressor(splitter='random', min_weight_fraction_leaf=0.1, min_samples_leaf=5, max_leaf_nodes=None, max_depth=9)
Dt.fit(X_train, y_train)
y_pred_dt = Dt.predict(X_val)

In [130]:
Gbr = GradientBoostingRegressor(learning_rate=0.16, max_depth=3, n_estimators=35)
Gbr.fit(X_train, y_train)
y_pred_gbr = Gbr.predict(X_val)

In [155]:
print("Mean Absolute error :", mean_absolute_error(y_val, y_pred_dt))
print("Root mean squared error :", mean_squared_error(y_val, y_pred_dt) ** 0.5)

Mean Absolute error : 1139.281536923879
Root mean squared error : 1508.0934123785587


In [152]:
print("Mean Absolute error :", mean_absolute_error(y_val, y_pred_gbr))
print("Root mean squared error :", mean_squared_error(y_val, y_pred_gbr) ** 0.5)

Mean Absolute error : 761.9310723027252
Root mean squared error : 1083.7904012551294


In [133]:
Final_model = GradientBoostingRegressor(learning_rate=0.16, max_depth=3, n_estimators=35)
Final_model.fit(X_train_df, Y_train_all)

GradientBoostingRegressor(learning_rate=0.16, n_estimators=35)

In [142]:
Final_model2 = DecisionTreeRegressor(splitter='best', min_weight_fraction_leaf=0.1, min_samples_leaf=4, max_leaf_nodes=60, max_features=None, max_depth=7)
Final_model2.fit(X_train_df, Y_train_all)

DecisionTreeRegressor(max_depth=7, max_leaf_nodes=60, min_samples_leaf=4,
                      min_weight_fraction_leaf=0.1)

In [134]:
final_predictions = Final_model.predict(X_test_df)

In [143]:
final_predictions2 = Final_model2.predict(X_test_df)

In [135]:
df_sol = pd.read_csv('sample_submission_8RXa3c6.csv')
df_submission = df_sol.copy()
df_submission['Item_Outlet_Sales'] = final_predictions
df_submission.to_csv('attempt16.csv', index=False)

In [144]:
df_sol = pd.read_csv('sample_submission_8RXa3c6.csv')
df_submission2 = df_sol.copy()
df_submission2['Item_Outlet_Sales'] = final_predictions2
df_submission2.to_csv('attempt18.csv', index=False)

In [145]:
(df_submission2['Item_Outlet_Sales']<=0).any()

False